In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import pickle

In [2]:
def reduce_mem_usage(df, verbose=False):
    '''
    reduce memory usage by downcasting data types
    from https://www.kaggle.com/harupy/m5-baseline
    '''
    
    start_mem = df.memory_usage().sum() / 1024 ** 2
    int_columns = df.select_dtypes(include=["int"]).columns
    float_columns = df.select_dtypes(include=["float"]).columns

    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")

    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [3]:
data=pd.read_pickle('/notebooks/final_data.pkl').pipe(reduce_mem_usage)
data.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,lag_21,lag_28,lag_30,lag_45,lag_60,lag_80,expanding_sold_mean,daily_avg_sold,avg_sold,selling_trend
6098000,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1501,0,2015-03-09,11506,...,0.0,0.0,2.0,0.0,2.0,0.0,0.646973,0.0,0.669434,-0.669434
6098001,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1501,1,2015-03-09,11506,...,0.0,0.0,0.0,0.0,0.0,1.0,0.318359,1.0,0.304932,0.695312


In [4]:
from tqdm import tqdm
for col in tqdm(['id','item_id','dept_id','cat_id','store_id','state_id','event_name_1','event_type_1','event_type_2','event_name_2','weekday']):
    data[col] = data[col].astype('category')

100%|██████████| 11/11 [00:00<00:00, 75.42it/s]


In [5]:
d_id = dict(zip(data.id.cat.codes, data.id))
d_item_id = dict(zip(data.item_id.cat.codes, data.item_id))
d_dept_id = dict(zip(data.dept_id.cat.codes, data.dept_id))
d_cat_id = dict(zip(data.cat_id.cat.codes, data.cat_id))
d_store_id = dict(zip(data.store_id.cat.codes, data.store_id))
d_state_id = dict(zip(data.state_id.cat.codes, data.state_id))

In [6]:
cols = data.dtypes.index.tolist()
d_types = data.dtypes.values.tolist()
for i,type in enumerate(d_types):
    if type.name == 'category':
        data[cols[i]] = data[cols[i]].cat.codes

In [7]:
data.drop(['date'],axis=1,inplace=True)

In [8]:
X_train, y_train = data[data['d']<1914].drop('demand',axis=1), data[data['d']<1914]['demand']
X_valid, y_valid = data[(data['d']>=1914) & (data['d']<1942)].drop('demand',axis=1), data[(data['d']>=1914) & (data['d']<1942)]['demand']
X_test = data[data['d']>=1942].drop('demand',axis=1)

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Flatten,Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
import csv
from tensorflow.keras.models import Model
#import keras
from tensorflow.keras import backend as k
from tensorflow.keras.layers import Input,Concatenate,Dropout,Dense,BatchNormalization,Conv1D
from tensorflow.keras.layers import Input
from tensorflow.keras.initializers import he_normal,glorot_normal
from tensorflow.keras.regularizers import l1,l2
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau
from time import time
from tensorflow.keras.utils import plot_model

In [16]:
def callback(filename):
    checkpoint = ModelCheckpoint(filename, monitor='val_root_mean_squared_error', verbose=1, mode='min')
    earlystop = EarlyStopping(monitor = 'val_root_mean_squared_error',  mode="min",min_delta = 0, patience = 3,verbose = 1)
    reduce_lr = ReduceLROnPlateau(monitor = 'val_root_mean_squared_error', factor = 0.25, patience = 2, verbose = 1)
    callbacks = [checkpoint, earlystop,reduce_lr]
    return callbacks

In [12]:
!pwd

/notebooks/Deep_Learning_model/LSTM_model_1


In [11]:
os.chdir('Deep_Learning_model/LSTM_model_1')

In [14]:
def Time_series_model_0(input_shape):
    model = Sequential()
    model.add(LSTM(units=150, return_sequences=True, input_shape = (input_shape,1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=25))
    model.add(Dense(1, activation='linear'))
    return model

In [ ]:
from tqdm import tqdm
import joblib
import gc
import warnings
warnings.filterwarnings('ignore')
evaluation=pd.DataFrame()
validation=pd.DataFrame()
valid_preds={}
eval_preds={}
data_new=pd.DataFrame()
stores = d_store_id.keys()
stores = d_store_id.keys()
for store in tqdm(stores):
    df = data[data['store_id']==store]
    print('********Prediction for Store:{}**********'.format(d_store_id[store]))
    X_train, y_train = df[df['d']<1914].drop('demand',axis=1), df[df['d']<1914]['demand']
    X_valid, y_valid = df[(df['d']>=1914) & (df['d']<1942)].drop('demand',axis=1), df[(df['d']>=1914) & (df['d']<1942)]['demand']
    X_test = df[df['d']>=1942].drop('demand',axis=1)
    model = Time_series_model_0(X_train.shape[1])
    filename = 'model_'+str(d_store_id[store])+'.pkl'
    model.compile( optimizer='adam',loss='mean_squared_error',metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.fit(x=X_train, y=y_train,epochs=50,verbose=1,batch_size=512, callbacks=callback(filename), validation_data=(X_valid, y_valid))
    y_pred_valid=model.predict(X_valid)
    y_pred_eval=model.predict(X_test)
    X_valid['demand']=y_pred_valid
    X_test['demand']=y_pred_eval
    X_valid=X_valid[['id','d','demand']]
    X_test=X_test[['id','d','demand']]
    validation = validation.append(X_valid)
    evaluation = evaluation.append(X_test)
    del X_train, y_train, y_valid, X_valid
    # save model
    #joblib.dump(model, filename)

  0%|          | 0/10 [00:00<?, ?it/s]

********Prediction for Store:CA_1**********
Epoch 1/50
2459/2460 [============================>.] - ETA: 0s - loss: 2.4220 - root_mean_squared_error: 1.5563
Epoch 1: saving model to model_CA_1.pkl


INFO:tensorflow:Assets written to: model_CA_1.pkl/assets


INFO:tensorflow:Assets written to: model_CA_1.pkl/assets


2460/2460 [==============================] - 75s 29ms/step - loss: 2.4215 - root_mean_squared_error: 1.5561 - val_loss: 0.6595 - val_root_mean_squared_error: 0.8121 - lr: 0.0010
Epoch 2/50
2459/2460 [============================>.] - ETA: 0s - loss: 0.5825 - root_mean_squared_error: 0.7632
Epoch 2: saving model to model_CA_1.pkl


...

INFO:tensorflow:Assets written to: model_TX_3.pkl/assets


2460/2460 [==============================] - 74s 30ms/step - loss: 0.0226 - root_mean_squared_error: 0.1504 - val_loss: 0.0063 - val_root_mean_squared_error: 0.0795 - lr: 6.2500e-05
Epoch 22/50
2459/2460 [============================>.] - ETA: 0s - loss: 0.0222 - root_mean_squared_error: 0.1490
Epoch 22: saving model to model_TX_3.pkl


INFO:tensorflow:Assets written to: model_TX_3.pkl/assets


INFO:tensorflow:Assets written to: model_TX_3.pkl/assets


2460/2460 [==============================] - 74s 30ms/step - loss: 0.0222 - root_mean_squared_error: 0.1490 - val_loss: 0.0064 - val_root_mean_squared_error: 0.0802 - lr: 6.2500e-05
Epoch 23/50
1312/2460 [===============>..............] - ETA: 28s - loss: 0.0129 - root_mean_squared_error: 0.1135